# Filter down the raw dataset to create the training dataset, with labels

In [1]:
import pandas as pd
import swifter
import numpy as np

In [2]:
y2017_data = pd.read_csv("../Datasets/2017_data.csv")

### Aggregate 2017 data

The data is originally split by both the census tract as well as the measurement (diabetes rate, obesity rate, etc)\
These rows should be aggregated into a single row, that contains several of those measurements

In [3]:
measure_ids = list(y2017_data['MeasureId'].unique())

## MeasureId's with their associated descriptions, in the 2017 data

In [4]:
dict(zip(y2017_data['MeasureId'], y2017_data['Measure']))

{'ARTHRITIS': 'Arthritis among adults aged >=18 Years',
 'CSMOKING': 'Current smoking among adults aged >=18 Years',
 'CHD': 'Coronary heart disease among adults aged >=18 Years',
 'OBESITY': 'Obesity among adults aged >=18 Years',
 'CHOLSCREEN': 'Cholesterol screening among adults aged >=18 Years',
 'BINGE': 'Binge drinking among adults aged >=18 Years',
 'COPD': 'Chronic obstructive pulmonary disease among adults aged >=18 Years',
 'DIABETES': 'Diagnosed diabetes among adults aged >=18 Years',
 'CASTHMA': 'Current asthma among adults aged >=18 Years',
 'ACCESS2': 'Current lack of health insurance among adults aged 18–64 Years',
 'KIDNEY': 'Chronic kidney disease among adults aged >=18 Years',
 'STROKE': 'Stroke among adults aged >=18 Years',
 'LPA': 'No leisure-time physical activity among adults aged >=18 Years',
 'BPHIGH': 'High blood pressure among adults aged >=18 Years',
 'CANCER': 'Cancer (excluding skin cancer) among adults aged >=18 Years',
 'CHECKUP': 'Visits to doctor for r

#### General location information. these should be the same across a single census tract

In [5]:
location_columns = [
    'StateAbbr',
    'StateDesc',
    'CityName',
    'CityFIPS',
    'TractFIPS',
]

In [6]:
# takes the result of the groupby, and combine into a single row
def aggregate_census_tract_data(census_tract_data):
    new_row = {}

    # add location information to new row
    for location_column in location_columns:
        new_row[location_column] = census_tract_data[location_column].values[0]
    
    # also get the 2017 population data. this can be used later for weighted averaging
    new_row['Population_2017'] = census_tract_data['PopulationCount'].values[0]
    
    # each row represents a specific measurement from this census tract
    for index, row in census_tract_data.iterrows():
        measure_id = row['MeasureId']
        measure_value = row['Data_Value']
        new_row[f'{measure_id}_2017'] = measure_value
    
    return new_row

In [7]:
row_list = list(y2017_data.groupby(['TractFIPS']).apply(aggregate_census_tract_data))

In [8]:
training_data = pd.DataFrame(row_list)

# FIPS code re-formatting
FIPS is how various government entities label areas. The data in raw csv format loses some information.\
IE 01125012502 becomes 1125012502.0\
Need to convert this back to string format, and parse out other codes:\

In [9]:
# original format
training_data[['CityFIPS', 'TractFIPS']].head(3)

,CityFIPS,TractFIPS
0,107000.0,1.073000e+09
1,107000.0,1.073000e+09
2,107000.0,1.073000e+09


In [10]:
# reformat the TractFIPS, adding back '0' at the front if necessary
def reformat_tract_fips(tract_fips):
    str_form = str(tract_fips).split('.')[0]
    return str_form.zfill(11) # zfill fills the left with 0's

training_data['TractFIPS'] = training_data['TractFIPS'].swifter.apply(reformat_tract_fips)

Pandas Apply:   0%|          | 0/27648 [00:00<?, ?it/s]

In [11]:
def reformat_city_fips(city_fips):
    str_form = str(city_fips).split('.')[0]
    return str_form.zfill(7) # zfill fills the left with 0's
training_data['CityFIPS'] = training_data['CityFIPS'].swifter.apply(reformat_city_fips)

Pandas Apply:   0%|          | 0/27648 [00:00<?, ?it/s]

In [12]:
# original format
training_data[['CityFIPS', 'TractFIPS']].head(3)

,CityFIPS,TractFIPS
0,0107000,01073000100
1,0107000,01073000300
2,0107000,01073000400


# FIPS code parsing

Now that the FIPS code is back in the normal string form, can parse out some information\
According to www.census.gov, the TractFIPS has 3 parts:\
- State Code (+2)
- County Code (+3)
- Tract Code (+6)

IE 01073000100 would become
- State Code "01"
- County Code "01073" (this will be used later to join with the poverty data)
- Tract Code "01073000100"

In [13]:
def get_county_fips(tract_fips):
    return tract_fips[0:5]

In [14]:
training_data['CountyFIPS'] = training_data['TractFIPS'].swifter.apply(get_county_fips)

Pandas Apply:   0%|          | 0/27648 [00:00<?, ?it/s]

In [15]:
training_data[['TractFIPS', 'CountyFIPS']].head(3)

,TractFIPS,CountyFIPS
0,01073000100,01073
1,01073000300,01073
2,01073000400,01073


In [16]:
county_fips_col = training_data.pop('CountyFIPS')
training_data.insert(3, 'CountyFIPS', county_fips_col)

In [17]:
training_data.head(3)

,StateAbbr,StateDesc,CityName,CountyFIPS,CityFIPS,TractFIPS,Population_2017,COPD_2017,STROKE_2017,LPA_2017,...,CASTHMA_2017,BPMED_2017,HIGHCHOL_2017,CSMOKING_2017,BINGE_2017,ARTHRITIS_2017,ACCESS2_2017,BPHIGH_2017,DIABETES_2017,PHLTH_2017
0,AL,Alabama,Birmingham,01073,0107000,01073000100,3042,11.3,5.5,44.0,...,13.4,77.8,35.3,27.9,10.3,31.2,24.4,47.7,17.4,20.5
1,AL,Alabama,Birmingham,01073,0107000,01073000300,2735,11.9,6.1,47.5,...,12.5,78.5,36.8,29.3,10.5,30.5,30.4,48.2,19.8,22.2
2,AL,Alabama,Birmingham,01073,0107000,01073000400,3338,10.5,5.6,43.0,...,13.3,79.4,35.6,25.9,10.4,31.5,24.7,49.1,18.4,19.4


# Check for nulls in the target variable (diabetes_change_rate)

In [18]:
# original number of rows
training_data.shape[0] 

27648

In [19]:
# keep only rows with the target variable
training_data = training_data.dropna()
training_data.shape[0]

26878

In [20]:
# lost about 700 rows after dropping nulls, should be fine
training_data.to_csv('../Datasets/training_data.csv', index=False)

In [21]:
training_data

,StateAbbr,StateDesc,CityName,CountyFIPS,CityFIPS,TractFIPS,Population_2017,COPD_2017,STROKE_2017,LPA_2017,...,CASTHMA_2017,BPMED_2017,HIGHCHOL_2017,CSMOKING_2017,BINGE_2017,ARTHRITIS_2017,ACCESS2_2017,BPHIGH_2017,DIABETES_2017,PHLTH_2017
0,AL,Alabama,Birmingham,01073,0107000,01073000100,3042,11.3,5.5,44.0,...,13.4,77.8,35.3,27.9,10.3,31.2,24.4,47.7,17.4,20.5
1,AL,Alabama,Birmingham,01073,0107000,01073000300,2735,11.9,6.1,47.5,...,12.5,78.5,36.8,29.3,10.5,30.5,30.4,48.2,19.8,22.2
2,AL,Alabama,Birmingham,01073,0107000,01073000400,3338,10.5,5.6,43.0,...,13.3,79.4,35.6,25.9,10.4,31.5,24.7,49.1,18.4,19.4
3,AL,Alabama,Birmingham,01073,0107000,01073000500,2864,12.3,7.3,45.9,...,13.6,82.1,38.4,26.3,9.1,35.0,25.2,54.1,22.4,21.7
4,AL,Alabama,Birmingham,01073,0107000,01073000700,2577,14.8,9.0,52.2,...,14.7,81.8,39.0,30.9,7.6,36.3,29.0,56.5,25.0,25.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27643,WY,Wyoming,Cheyenne,56021,5613900,56021001401,3961,5.8,2.5,24.4,...,9.5,70.7,30.5,18.0,17.9,22.9,14.7,29.0,8.1,10.4
27644,WY,Wyoming,Cheyenne,56021,5613900,56021001402,1913,6.2,3.3,24.7,...,9.2,76.9,32.5,14.2,15.1,27.3,13.6,33.1,8.8,10.5
27645,WY,Wyoming,Cheyenne,56021,5613900,56021001501,3312,5.3,2.3,23.7,...,9.4,69.9,29.1,17.4,18.4,22.0,14.5,27.8,7.4,9.6
27646,WY,Wyoming,Cheyenne,56021,5613900,56021001502,4518,7.0,3.2,28.2,...,9.9,72.9,31.9,19.8,16.0,25.2,16.6,31.9,9.4,12.0
